## Import the Segmented Corpus

In [ ]:
# If these packages are not installed:

# ! pip install git+https://github.com/iinemo/isanlp.git
# ! pip install isanlp_rst
# ! pip install hf_xet

In [ ]:
# === Import
import pandas as pd
import re
import sys
import json
from pathlib import Path

import warnings
warnings.filterwarnings("ignore", message="`encoder_attention_mask` is deprecated")

# === Define the path to the auxiliary modules ===
ROOT = Path.cwd().parent
SRC = (ROOT / "src").resolve()

if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# === import the module for rst work === <--- this is not ready yet
# from discourse.rst import init_parser # this is not ready yet

In [ ]:
# === Define the path to the data and the pattern for retrieval ==
HOME = Path.home()
DATA_DIR = (HOME / "My Drive" / "_VectorData" / "projects" / "identifying_depression_with_rst" / "data").resolve(strict=True)

corpus_path = DATA_DIR / "processed"
corpus_file = corpus_path / "segmented_corpora_320_token_split.json"

with open(corpus_file, "r") as file:
    corpora = json.load(file)

In [ ]:
# === This is just in case for possible debugging (with verbose output) ===
# import transformers
# transformers.utils.logging.set_verbosity_info()
# transformers.utils.logging.enable_explicit_format()

## Visual Inspection

In [ ]:
# === A note on the structure of the resulting segmented corpora: ===
# Each separate corpus in the corpora is the value for the key indicating the name of this sepcific corpus (like 'ked' in this case)

corpora.keys()

In [ ]:
# === Further down, the tree the structure is as follows: ===
# The value of the key is a list of 2 items
# Where each item is also a list
# The first list is the original text either as a single list item if it has not been split
# Or as several itmes, which are the resulting chunks of the splitting pipeline upstream
# The second list is made up of the sentences returned by the sentence tokenizer as list items

# So, pulling the text (or the resulting chunks) for the "ked" coprus looks something like this:

corpora["ked"][0][0]

In [ ]:
len(corpora["ked"])

## Prep Everything for Running the RST Parser

In [ ]:
# Using the parser from the notebook directly for now (while the rst module is WIP)
# This will later be handled by the rst.py module

from isanlp_rst.parser import Parser

model = 'tchewik/isanlp_rst_v3'
version = 'gumrrg'  # Choose from {'gumrrg', 'rstdt', 'rstreebank'}

parser = Parser(hf_model_name=model, hf_model_version=version)

In [ ]:
# Put the target corpus as texts/chunks into a separate variable for easier navigation/iteration logic downstream

corpus_name = "ked"

corpus = [item[0] for item in corpora[corpus_name]] # grab only the texts/segments, not the texts as sentences

In [ ]:
corpus[0]

In [ ]:
# Check that the nuber of segmented texts matches what we got in the previous stage (segmentation)

find_split_texts = [i for i in corpus if len(i) > 1]
len(find_split_texts)

In [ ]:
# === This works on the specific corpus from the corpora: ===

# The code expects the text either as one solid chunk or as two or more chunks
# in case the text has been segmented in the previous phase (segmentation)


def _as_segments(x):
    # normalize each item in corpus to a list[str] for cases where the structure of the corpus may be like
    # ["text1", ["seg1_of_text2", "seg2_of_text2"], "text3", ... ]
    # this is just an extra precaution
    
    if isinstance(x, str):
        return [x]
    return [s.strip() for s in x if isinstance(s, str) and s.strip()]

In [ ]:
# === A helper to 'normalize' the texts just in case
import re, unicodedata

def normalize_for_parser(s: str) -> str:
    s = s.replace("\u00A0", " ")        # NBSP -> space
    s = s.replace("\u200b", "")         # zero-width space
    s = s.replace("\ufeff", "")         # BOM
    s = re.sub(r"\r\n?", "\n", s)       # CRLF/CR -> LF
    s = unicodedata.normalize("NFC", s) # canonical compose
    return s.strip()

## Parsing the Selected Corpus

In [ ]:
parsed_corpus = []
errors = []

for di, doc in enumerate(corpus):
    segments = _as_segments(doc)
    parsed_segments = []
    for si, seg in enumerate(segments):
        try:
            print(f"processing doc:{di}, segment: {si}", flush=True)
            parsed_segments.append(parser(normalize_for_parser(seg)))
        except Exception as e:
            errors.append({"doc_index": di, "seg_index": si, "error": str(e)})
            parsed_segments.append(None)  # keeping alignment helps
    parsed_corpus.append(parsed_segments)


In [ ]:
len(parsed_corpus)

## A Bit of Visual Inspection

In [ ]:
# === What does the parser return and how is it structured in our 'segmented' corpus?

# First, each item is a list of at least one item -- a dictionary returned by the parser:
parsed_corpus[0]

In [ ]:
# Further down the tree, each such dictionary has the key "rst", which stores the results of parsing as a list of one item (in our case)
# This item is the RST object/tree proper
# It can be explored using the 'vars' function:

vars(parsed_corpus[0][0]['rst'][0])

## Investigate Why A Segment is Reported to be over 512

In [ ]:
import logging, transformers
transformers.utils.logging.set_verbosity_info()
transformers.utils.logging.enable_explicit_format()

parsed_corpus, offenders = [], []
for di, doc in enumerate(corpus):
    for si, seg in enumerate(_as_segments(doc)):
        print(f"processing doc:{di}, seg:{si}", flush=True)
        try:
            _ = parser(seg)                 # just probe once
            parsed_corpus.append(None)      # we're not keeping results here
        except Exception as e:
            offenders.append((di, si, str(e)))


In [ ]:
from transformers import AutoTokenizer

# model_name = "DeepPavlov/rubert-base-cased"
# model_name = "ai-forever/ruBert-base"
model_name = "sberbank-ai/ruBert-large"


tok = AutoTokenizer.from_pretrained(model_name)
for item in corpus:
    for segment in item:
        print(len(tok(normalize_for_parser(segment), add_special_tokens=True)["input_ids"]))

In [ ]:
res_temp = parser(normalize_for_parser(corpus[106][2]))

In [ ]:
import re, unicodedata
from transformers import AutoTokenizer

# 1) close-enough tokenizer to what the RST parser likely uses
xlmr = AutoTokenizer.from_pretrained("xlm-roberta-base")

def single_len(text: str) -> int:
    return len(xlmr(text, add_special_tokens=True)["input_ids"])

def pair_len(a: str, b: str) -> int:
    # this mimics how RoBERTa encodes pairs: <s>a</s></s>b</s>
    return len(xlmr(a, b, add_special_tokens=True)["input_ids"])

def max_pair_len(text: str) -> int:
    # super-naive sentence split is fine for triage; plug in your splitter if you want
    sents = [t for t in re.split(r'(?<=[.!?])\s+', text) if t.strip()]
    m = 0
    for i in range(len(sents)-1):
        m = max(m, pair_len(sents[i], sents[i+1]))
        if m >= 512:  # early exit
            break
    return m

offenders = []
for di, doc in enumerate(corpus):
    for si, raw in enumerate(_as_segments(doc)):
        seg = normalize_for_parser(raw)
        s_len = single_len(seg)
        p_len = max_pair_len(seg)
        if s_len >= 512 or p_len >= 512 or p_len >= 508:   # 508≈practical tripwire
            offenders.append((di, si, s_len, p_len))

print(f"Found {len(offenders)} risky segments (doc_idx, seg_idx, single_len, max_pair_len):")
for row in offenders[:20]:
    print(row)

## NEXT:
* Walk the RST Trees and Extract the Data (this part is WIP)